> 1-1) x & y data split (최종적으로 예측해야할 대상인 survived 열은 age 예측에 활용되면 안됩니다)
> 
> 1-2) train & test split (test 데이터의 행들은 age 열을 예측하는 ML 모델 훈련 시 활용되면 안됩니다.)
>
> 2) train_x 기준으로 기존 index 열을 일반 열로 만들어줍니다. 일반 열이 된 기존 index 열은 (y 데이터와의 행 기준 순서를 일치시키기 위해) 나중에 age열 결측치를 채운 후 다시 df을 합친 다음 재정렬할 때 활용합니다.
>
>     : df.reset_index()
>
> 3) 일반 열이 된 기존 index 열을 포함한 train_x 데이터프레임을 age 열을 기준으로 결측치가 있는 행들과 결측치가 없는 행들로 나눠 2개의 데이터프레임으로 쪼갭니다.
> 
>     ex)
>     x_train['Age'] = x_train['Age'].fillna(0)
>     x_train_age_null = x_train[ x_data['Age'] == 0 ] 
>     x_train_age_notnull = x_train[ x_data['Age'] != 0 ] 
>    
> 4-1) age 열에 결측치가 없는 행들의 df 을 기준으로 하여, age 열과 기존 index 열을 제외한 나머지 열들을 train_x 로, age 열을 train_y 로 쪼갭니다.
>
> 4-2) age 열에 결측치가 있는 행들의 df 을 기준으로 하여, age 열과 기존 index 열을 제외한 나머지 열들을 test_x 로, age 열을 test_y 로 쪼갭니다.
>
> 5-1) 위에서 만든 train_x 로 train_y (age) 를 맞추는 ML 모델을 만들고 학습시킵니다.
>
> 5-2) 위에서 만든 ML 모델로 test_x를 대상으로 age를 predict 하여 별도의 변수로 만듭니다 (ex. predicted_age)
>
> 6-1) age 열에 결측치가 있는 행들의 df 을 기준으로 하여, 위에서 만든 예측된 나이값(ex. predicted_age)을 가져와 age 열을 덮어씌웁니다.
>
> 6-2) age 열에 결측치가 없는 행들의 df 와 age 열에 결측치가 있었던(이제는 채워진) 행들의 df 를 위아래로 합칩니다. (pd.concat 활용)
>
> 7-1) survived 열과의 행 기준 순서를 일치시키기 위해, 일반 열로 만들어두었던 기존 index 열을 기준으로 재정렬합니다. (sort_values)
>
> 7-2) 일반 열로 만들어두었던 기존 index 열을 제거합니다.
>
> 8) 위 1-2) 에서 나눠두었던 test data를 기준으로 age 열의 값을 채워넣는 작업을 적용합니다.
>
> - 기존 index 열을 일반 열로 변경
> - age 열에 결측치가 있는 행들만 모아 df 로 구성
> - age 열과 기존 index 열을 제외한 나머지 열들을 x 로 하여 age를 예측하고 age 열에 덮어씌움
> - 결측치가 채워진 상태에서 결측치가 없던 행들의 df 와 위아래로 합침
> - 기존 index 열 기준으로 재정렬 & 기존 index 열 삭제
> 
> 9) survived 열을 맞추기 위한 모델을 만듭니다.

위와 같은 강사님의 방법을 구현

### 0-1) Import

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn import linear_model, svm, ensemble
from xgboost import XGBRegressor

from vecstack import StackingTransformer

from collections import Counter

from tqdm import tqdm

### 0-2) 데이터 준비

In [2]:
data_df = pd.read_csv('../titanic.csv')

In [3]:
deck_df = pd.read_pickle('../deck_df.pkl')

In [4]:
data_df = data_df.merge(deck_df, left_index=True, right_index=True)\
            [['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'deck']]

In [5]:
data_df = data_df

### 1-1) x  & y Split

In [6]:
x_data = data_df.copy().drop('Survived', axis=1)
y_data = data_df.Survived

### 1-2) train / test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, shuffle=True, random_state=2, test_size=0.3)

### 2) 기존 index를 일반열로 전환

In [8]:
X_train.reset_index(inplace=True)
X_train

,index,Pclass,Age,SibSp,Parch,Fare,Embarked,deck
0,451,3,NaN,1,0,19.9667,S,3
1,345,2,24.0,0,0,13.0000,S,6
2,687,3,19.0,0,0,10.1708,S,5
3,279,3,35.0,1,1,20.2500,S,6
4,742,1,21.0,2,2,262.3750,C,2
...,...,...,...,...,...,...,...,...
618,534,3,30.0,0,0,8.6625,S,6
619,584,3,NaN,0,0,8.7125,C,4
620,493,1,71.0,0,0,49.5042,C,5
621,527,1,NaN,0,0,221.7792,S,3


In [9]:
# X_train.set_index('index', inplace=True)
# X_train

### 3) 결측치 분리

In [10]:
age_null_list = []
age_not_null_list = []

for row in tqdm(X_train.itertuples()):
    if(pd.isna(row.Age)):
        age_null_list.append(row[0])
    else:
        age_not_null_list.append(row[0])

623it [00:00, 306408.47it/s]


In [11]:
# age_not_null_list

In [12]:
X_train_age_null = X_train.iloc[ age_null_list ]
X_train_age_notnull = X_train.iloc[ age_not_null_list ] 

### 4-1) 결측치가 없는 행 Train / Test Split

In [13]:
X_age_train = X_train_age_notnull.drop('Age', axis=1)
y_age_train = X_train_age_notnull['Age']

### 4-2) 결측치가 있는 행 Train / Test Split

In [14]:
X_age_test = X_train_age_null.drop('Age', axis=1)
y_age_test = X_train_age_null['Age']

### 5-1) 모델 객체 생성 & 학습

#### Pipeline Preprocess

In [15]:
# 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'deck'
feature_cols = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked', 'deck']
target_col = 'Age'

numeric_features = ['SibSp', 'Parch', 'Fare']
numeric_transformer = RobustScaler() # StandardScaler()

categorical_features = ['Pclass', 'Embarked', 'deck']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') 

preprocessor = ColumnTransformer(
    transformers=[ # List of (name, transformer, column(s))
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [16]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

preprocessor_pipe.fit(pd.DataFrame(X_age_train, columns=feature_cols))

X_train_transformed = preprocessor_pipe.transform(pd.DataFrame(X_age_train, columns=feature_cols))
X_test_transformed = preprocessor_pipe.transform(pd.DataFrame(X_age_test, columns=feature_cols))

#### Grid Search

In [17]:
param_grid = {'C' : [0.1, 1, 10, 100, 1000], 
             'gamma' : [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel' : ['rbf']}

grid = GridSearchCV(svm.SVR(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.1, gamma=0.0

[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ...................C=1000, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ...................C=1000, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ...................

In [18]:
param_grid = {'C' : [0.1, 1, 10, 100, 1000]}

grid = GridSearchCV(svm.LinearSVR(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ..............................................C=0.1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.0s
[CV] END ................................................C=1; total time=   0.0s
[CV] END ...............................................C=10; total time=   0.0s
[CV] END ........................................

In [19]:
param_grid = {
    'n_estimators' : [100, 200, 500, 1000]
    , 'n_jobs' : [-1]
    , 'max_depth' : [3, 4, 5, 8]
}

grid = GridSearchCV(ensemble.ExtraTreesRegressor(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   5.1s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=500, n_jobs=-1; total time=   0.3s
[CV] END ...........max_depth=3, n_estimators=50

In [20]:
param_grid = {
    'n_estimators' : [100, 200, 500, 1000]
    , 'n_jobs' : [-1]
    , 'max_depth' : [3, 4, 5, 8]
}

grid = GridSearchCV(ensemble.RandomForestRegressor(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.0s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=500, n_jobs=-1; total time=   0.4s
[CV] END ...........max_depth=3, n_estimators=50

In [21]:
param_grid = {
    'n_estimators' : [100, 200, 500]
    , 'max_depth' : [3, 4, 5]
    , 'learning_rate' : [0.001, 0.01, 0.05, 0.1]
}

grid = GridSearchCV(ensemble.GradientBoostingRegressor(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.0s
[CV] END .learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.1s
[CV] END .learning_rate=0.001, max_depth=3, n_e

[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.1s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.1s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.1s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.0s
[CV] END ..learning_rate=0.0

In [22]:
param_grid = {
    'n_estimators' : [200, 500, 1000]
    , 'gamma' : [1, 0.1, 0.01, 0.001]
    , 'max_depth' : [3, 4, 5]
    , 'learning_rate' : [0.001, 0.01, 0.05, 0.1]
}

grid = GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=2)

grid.fit(X_train_transformed, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=1, learning_rate=0.001, max_dept

[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=1, learning_ra

[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.7s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.1, learning_rate=0.001, max_depth=4, n_estimators=200; total time=  

[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.1, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] 

[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.4s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.001, max_depth=4, n_estimators=200; to

[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.01, learning_rate=0.05, max_depth=4, n_estimators=200; total time=  

[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=500; total time=   0.8s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.001, max_depth=4, n_estimat

[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=500; total time=   0.7s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.5s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=3, n_estimators=1000; total time=   1.6s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.3s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=4, n_estimators=200; total time=   0.4s
[CV] END gamma=0.001, learning_rate=0.05, max_depth=4, n_estimators=200; to

#### Stacking Transformer 객체 생성 & 학습

In [23]:
estimators = [ 
    ('ExtraTrees', ensemble.ExtraTreesRegressor(random_state=0, n_jobs=-1, n_estimators=500, max_depth=5))
    , ('RandomForest', ensemble.RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=100, max_depth=4))
    , ('GB', ensemble.GradientBoostingRegressor(n_estimators=200, max_depth=3, learning_rate=0.01))
    , ('XGB', XGBRegressor(n_jobs = -1, gamma=0.01, learning_rate = 0.01, n_estimators = 500, max_depth = 3))
    , ('SVR', svm.SVR(C=10, gamma=.1, kernel='rbf'))
    , ('LSVR', svm.LinearSVR(C=10, random_state=0))
    , ('LinearReg', linear_model.LinearRegression(n_jobs=-1))
]

In [24]:
stack = StackingTransformer(estimators, 
                            regression = True, 
                            metric = mean_squared_error, 
                            n_folds = 4, shuffle = True, 
                            random_state = 0, verbose = 1)

In [25]:
stack = stack.fit(X_train_transformed, y_age_train)

task:         [regression]
metric:       [mean_squared_error]
variant:      [A]
n_estimators: [7]

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    MEAN:     [154.04168076] + [33.07180011]

estimator  1: [RandomForest: RandomForestRegressor]
    MEAN:     [147.44294778] + [28.75463248]

estimator  2: [GB: GradientBoostingRegressor]
    MEAN:     [147.78937402] + [31.45075581]

estimator  3: [XGB: XGBRegressor]
    MEAN:     [150.86190861] + [28.16100211]

estimator  4: [SVR: SVR]
    MEAN:     [156.61020486] + [31.44801610]

estimator  5: [LSVR: LinearSVR]
    MEAN:     [171.77059722] + [36.81923332]

estimator  6: [LinearReg: LinearRegression]
    MEAN:     [165.67281277] + [37.54554766]



#### Stacked Feature

In [26]:
S_train = stack.transform(X_train_transformed)
S_test = stack.transform(X_test_transformed)

Train set was detected.
Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE



#### Second Level Estimator

##### Set Hyper Parameter

가장 에러가 적었던 RandomForest 선정

In [27]:
param_grid = {
    'n_estimators' : [100, 200, 500]
    , 'n_jobs' : [-1]
    , 'max_depth' : [3, 4, 5]
}

grid = GridSearchCV(ensemble.RandomForestRegressor(), param_grid, refit=True, verbose=2)

grid.fit(S_train, y_age_train)
print('The best parameters are ', grid.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   9.3s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=100, n_jobs=-1; total time=   0.1s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.2s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.2s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.2s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.3s
[CV] END ...........max_depth=3, n_estimators=200, n_jobs=-1; total time=   0.3s
[CV] END ...........max_depth=3, n_estimators=500, n_jobs=-1; total time=   0.8s
[CV] END ...........max_depth=3, n_estimators=500

##### 모델 객체 생성 & 학습

In [28]:
model = ensemble.RandomForestRegressor(max_depth=3, n_estimators=200, n_jobs=-1)

model = model.fit(S_train, y_age_train)

print(mean_squared_error(model.predict(S_train), y_age_train))

122.02976509229943


1차 모델 각각의 결과 보다 2차 모델 결과가 에러가 더 적다

### 5-2) 나이 예측

In [29]:
age_list = []

for row in data_df.itertuples():
    if(pd.isna(row.Age)):
        tmp_X_data = np.array([row.Pclass, row.SibSp, row.Parch, row.Fare, row.Embarked, row.deck])
        tmp_X_data = pd.DataFrame([tmp_X_data], columns=['Pclass', 'SibSp', 'Parch', 'Fare', 'Embarked', 'deck'])
        tmp_X_data = preprocessor_pipe.transform(tmp_X_data)
        tmp_X_data = stack.transform(tmp_X_data)
        age_list.append(model.predict(tmp_X_data))
    else:
        age_list.append(row.Age)

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
  

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [LinearReg: LinearRegression]
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesRegressor]
    DONE

estimator  1: [RandomForest: RandomForestRegressor]
    DONE

estimator  2: [GB: GradientBoostingRegressor]
    DONE

estimator  3: [XGB: XGBRegressor]
    DONE

estimator  4: [SVR: SVR]
    DONE

estimator  5: [LSVR: LinearSVR]
    DONE

estimator  6: [

### 6~8) 예측 나이 저장

In [30]:
age_df = pd.DataFrame(age_list, columns=['EstAge'])
age_df['EstAge'] = age_df['EstAge'].astype(int)

In [31]:
age_df.to_pickle('../age_df.pkl')